# SVM

Thời lượng ước tính: **45** phút

## Mục tiêu

Sau khi hoàn thành lab này, bạn sẽ có thể:

-   Sử dụng scikit-learning vào SVM để phân loại


Trong notebook này, bạn sẽ sử dụng SVM (Máy véc-tơ hỗ trợ) để xây dựng và huấn luyện một mô hình sử dụng hồ sơ tế bào người, đồng thời phân loại tế bào theo mẫu là lành tính hay ác tính.

SVM hoạt động bằng cách ánh xạ dữ liệu tới không gian thuộc tính chiều cao để các điểm dữ liệu có thể được phân loại, ngay cả khi dữ liệu không được phân tách tuyến tính. Dấu phân cách giữa các danh mục được tìm thấy, sau đó dữ liệu được chuyển đổi theo cách mà dấu phân tách có thể được vẽ như một siêu phẳng. Sau đó,có thể sử dụng các đặc điểm của dữ liệu mới để dự đoán nhóm chứa bản ghi mới.


<h1>Mục lục</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#load_dataset">Load dữ liệu Ung thư</a></li>
        <li><a href="#modeling">Mô hình hóa</a></li>
        <li><a href="#evaluation">Đánh giá</a></li>
        <li><a href="#practice">Thực hành</a></li>
    </ol>
</div>
<br>
<hr>


In [ ]:
!pip install scikit-learn==0.23.1

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline 
import matplotlib.pyplot as plt

### Load dữ liệu từ tập CSV


In [ ]:
cell_df = pd.read_csv("cell_samples.csv")
cell_df.head()

Trường ID chứa các số nhận dạng bệnh nhân. Các đặc điểm của mẫu tế bào từ mỗi bệnh nhân được chứa trong các trường Clump tới Mit. Các giá trị được xếp loại từ 1 đến 10, với 1 là giá trị gần nhất với lành tính.

Trường Class chứa chẩn đoán, như được xác nhận bởi các thủ tục y tế riêng biệt về việc các mẫu là lành tính (giá trị = 2) hay ác tính (giá trị = 4).

Hãy xem sự phân bố của các lớp dựa trên độ dày Clump của cụm và tính đồng nhất của kích thước cell:

In [ ]:
ax = cell_df[cell_df['Class'] == 4][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='DarkBlue', label='malignant');
cell_df[cell_df['Class'] == 2][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='Yellow', label='benign', ax=ax);
plt.show()

## Tiền xử lý và chuẩn bị dữ liệu


Đầu tiên, hãy xem các kiểu dữ liệu cột:


In [ ]:
cell_df.dtypes

Có vẻ như cột **BareNuc** có một số giá trị không phải là số. Chúng ta có thể bỏ các hàng đó:


In [ ]:
cell_df = cell_df[pd.to_numeric(cell_df['BareNuc'], errors='coerce').notnull()]
cell_df['BareNuc'] = cell_df['BareNuc'].astype('int')
cell_df.dtypes

In [ ]:
feature_df = cell_df[['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize', 'BareNuc', 'BlandChrom', 'NormNucl', 'Mit']]
X = np.asarray(feature_df)
X[0:5]


Chúng ta muốn mô hình dự đoán giá trị của Class (lành tính (= 2) hoặc ác tính (= 4)). Vì trường này chỉ có thể có một trong hai giá trị, nên chúng ta cần thay đổi mức đo lường của nó để phản ánh điều này.


In [ ]:
cell_df['Class'] = cell_df['Class'].astype('int')
y = np.asarray(cell_df['Class'])
y [0:5]

## Tập dữ liệu Train/Test

Được rồi, hãy chia tập dữ liệu thành train set và test set:


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

<h2 id="modeling">Mô hình hóa (SVM với Scikit-learn)</h2>


Thuật toán SVM cung cấp lựa chọn các hàm nhân (kernel function) để thực hiện quá trình xử lý. Về cơ bản, ánh xạ dữ liệu vào một không gian có chiều cao hơn được gọi là kernelling. Hàm toán học sử dụng cho phép biến đổi được gọi là hàm nhân và có thể có nhiều loại khác nhau, chẳng hạn như:

```
1.Tuyến tính
2.Đa thức
3.Hàm cơ sở bán kính (Radial basis function - RBF)
4.Sigmoid
```

Mỗi hàm này đều có các đặc điểm, ưu nhược điểm và phương trình, nhưng khó biết hàm nào hoạt động tốt nhất với bất kỳ tập dữ liệu nhất định nào, chúng ta thường lần lượt chọn các hàm khác nhau và so sánh kết quả. Hãy chỉ sử dụng mặc định RBF (Radial Basis Function) cho lab này.


In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train) 

Sau khi được trang bị, có thể sử dụng mô hình sau để dự đoán các giá trị mới:


In [ ]:
yhat = clf.predict(X_test)
yhat [0:5]

<h2 id="evaluation">Đánh giá</h2>


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Tính confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat, labels=[2,4])
np.set_printoptions(precision=2)

print (classification_report(y_test, yhat))

# Vẽ non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Benign(2)','Malignant(4)'],normalize= False,  title='Confusion matrix')

Bạn cũng có thể dễ dàng sử dụng **f1_score**  từ thư viện sklearn:


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, yhat, average='weighted') 

Hãy thử jaccard index để biết độ chính xác:


In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, yhat,pos_label=2)

<h2 id="practice">Thực hành</h2>
Bạn có thể xây dựng lại mô hình, nhưng lần này với nhân __linear__ không? Bạn có thể sử dụng tùy chọn __kernel='linear'__ khi bạn xác định svm. Độ chính xác thay đổi như thế nào với hàm kernel mới?

In [ ]:
# Nhập code của bạn ở đây


<details><summary>Click vào đây để xem lời giải</summary>

```python
clf2 = svm.SVC(kernel='linear')
clf2.fit(X_train, y_train) 
yhat2 = clf2.predict(X_test)
print("Avg F1-score: %.4f" % f1_score(y_test, yhat2, average='weighted'))
print("Jaccard score: %.4f" % jaccard_score(y_test, yhat2,pos_label=2))

```

</details>


### Cảm ơn bạn đã hoàn thành lab này!

Nguồn bài Lab: **IBM**